# Push the model's prediction to Kaggle

### Load the model and the data


In [ ]:
import pickle
import numpy as np
import pandas as pd
from lightning.pytorch import Trainer
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from dmsensei.core import DataModule

model = pickle.load(open('model.pkl','rb'))
trainer = Trainer()

### Run the prediction

In [ ]:
prediction = trainer.predict(
        model,
        datamodule=DataModule(
            name=["ribo-test"],
            data='sequence',
            force_download=False,
            batch_size=256,
            num_workers=1,
            train_split=0,
            valid_split=0,
            predict_split=1.,
            overfit_mode=False,
        ),
    )

### Format the prediction and save it locally

In [ ]:
def format_to_ribonanza(prediction):
    prediction = np.vstack([np.array(a).reshape(-1, 2).round(3) for a in prediction])
    return pd.DataFrame(prediction, columns=["reactivity_DMS_MaP", "reactivity_2A3_MaP"]).reset_index().rename(columns={"index": "id"})

format_to_ribonanza(prediction).to_csv('submission.csv', index=False)

### Push the prediction to Kaggle

Note: setup your Kaggle authentification first:
1. download your Kaggle API keys `kaggle.json` here: https://www.kaggle.com/settings/account
2. save it to `~/.kaggle/kaggle.json`:

    ```bash
    mv ~/Downloads/kaggle.json ~/.kaggle/kaggle.json
    ```
3. push your results to kaggle using the following line

In [ ]:
os.system('kaggle competitions submit -c stanford-ribonanza-rna-folding -f out.csv -m "test commit"')